## Project Name - Indoor Localization using Naive Bayes classification using only cellular strengths first and then adding a few Wi-Fi signals to improve the accuracy. 
- Author - Anshul Roonwal                             
- Date - December 14th, 2015
- Course - CSE 570 Wireless and Mobile Communication


# Parsing the Data begins here 

In [31]:
import os
import re
import string
import pandas as pd
import numpy as np

dictDistinctCells = {}; dictDistinctWifi ={}
def getMaxCounts(directoryPathOfInputFiles):
    total_records=0; local_count_cell=0; local_count_wifi=0; max_cell_count=0; max_wifi_count=0
    for eachfile in os.listdir(directoryPathOfInputFiles):
        localRecords =0
        labels = eachfile.split("_")[0]
        
        fin = open(directoryPathOfInputFiles + eachfile,"r")
        next(fin);                                          #Skips first lines 
        for line in fin:
            splittedLine = re.split('\n', line)         
            if splittedLine[0].find('_') != -1 and re.match(r'[0-9]+_[0-9]+_[0-9]+', splittedLine[0]):         
                next(fin); next(fin); next(fin)
                if local_count_cell > max_cell_count:
                    max_cell_count = local_count_cell
                if local_count_wifi > max_wifi_count:
                    max_wifi_count = local_count_wifi
                local_count_cell=0; local_count_wifi=0  
                total_records = total_records+1         # Keeping track of useful rows
                localRecords = localRecords +1          
                continue                            
                
            if splittedLine[0].find("EDGE Cell") != -1:
                arr = splittedLine[0].split(' ')
                if arr[3] ==  '65535':                 ## We will ignore the Signals coming from 65535 
                    continue
                if arr[3] not in dictDistinctCells:
                    dictDistinctCells[arr[3]] = 1
                else:
                    dictDistinctCells[arr[3]] = dictDistinctCells[arr[3]] + 1
                local_count_cell = local_count_cell + 1    
        
            if splittedLine[0].find("WiFi:") != -1:
                arr = splittedLine[0].split(' ')
                if arr[1] not in dictDistinctWifi:
                    dictDistinctWifi[arr[1]] = 1
                else:
                    dictDistinctWifi[arr[1]] = dictDistinctWifi[arr[1]] + 1
                local_count_wifi= local_count_wifi+1    
        fin.close()        
    print 'Max Cell Count --> ', max_cell_count   
    print 'Max wifi Count --> ', max_wifi_count   
    print 'Total records -->', total_records      
    print 'Distinct Cells are-->', len(dictDistinctCells)
    print 'Distinct Wifi are-->', len(dictDistinctWifi)
    print '\n Distinct Cell Ids with count\n', dictDistinctCells, "\n"
    print '\n Distinct Wi-Fi Mac Ids with count\n', dictDistinctWifi, "\n"
    
    return max_cell_count, max_wifi_count, total_records

# You might have to change the path as per your system.
max_cell_count, max_wifi_count, total_records = getMaxCounts("C:/Users/Anshul/Desktop/Courses/Wireless/Final_Project/Initial_Data/")

Max Cell Count -->  6
Max wifi Count -->  31
Total records --> 3380
Distinct Cells are--> 12
Distinct Wifi are--> 53

 Distinct Cell Ids with count
{'8363': 1380, '8362': 1010, '7837': 13, '5273': 13, '7838': 2992, '38173': 4, '7001': 833, '61148': 110, '8411': 24, '7002': 1209, '6231': 412, '6232': 2369} 


 Distinct Wi-Fi Mac Ids with count
{'18:64:72:36:12:e4': 1392, '18:64:72:36:12:e2': 1772, '18:64:72:36:12:e3': 1814, '18:64:72:36:12:e0': 1330, '18:64:72:36:12:e1': 1788, '18:64:72:1d:90:40': 33, '18:64:72:1d:80:a4': 2319, '18:64:72:1d:80:a1': 2773, '18:64:72:1d:80:a0': 1869, '18:64:72:1d:80:a3': 2830, '18:64:72:1d:80:a2': 2749, 'fc:15:b4:d4:53:7c': 440, '18:64:72:1d:84:21': 2978, '18:64:72:1d:90:43': 699, '18:64:72:1d:90:42': 558, '18:64:72:1d:90:41': 636, 'f4:06:69:7d:3a:e3': 16, 'e8:cc:18:de:56:0c': 667, '18:64:72:1d:90:44': 383, '18:64:72:23:02:e1': 339, '18:64:72:23:02:e0': 5, '18:64:72:23:02:e3': 472, '18:64:72:23:02:e2': 383, '18:64:72:23:02:e4': 262, '18:64:72:1d:79:44': 5,

## Make column Headers
- Note- There will be as many columns as there are distinct Cell Ids and Wi-Fi Mac Ids recorded at the location of Data collection. Besides this, one last label for 'Label/Class'

In [8]:
#Make the column list first, now u know how many Distict cells and wifi could be there
def makeColumnHeaders(dictDistinctCells, dictDistinctWifi):
    newColumns = []
    for key, value in dictDistinctCells.iteritems():
        newColumns.append(key)
    for key, value in dictDistinctWifi.iteritems():
        newColumns.append(key)

    newColumns.append('label')
    return newColumns

newColumns = makeColumnHeaders(dictDistinctCells, dictDistinctWifi)
print 'Here is the list of all columns based on the distinct Cell Ids and Wi-Fi Ids + the label column in the end\n', newColumns

Here is the list of all columns based on the distinct Cell Ids and Wi-Fi Ids + the label column in the end
['8363', '8362', '7837', '5273', '7838', '38173', '7001', '61148', '8411', '7002', '6231', '6232', '18:64:72:36:12:e4', '18:64:72:36:12:e2', '18:64:72:36:12:e3', '18:64:72:36:12:e0', '18:64:72:36:12:e1', '18:64:72:1d:90:40', '18:64:72:1d:80:a4', '18:64:72:1d:80:a1', '18:64:72:1d:80:a0', '18:64:72:1d:80:a3', '18:64:72:1d:80:a2', 'fc:15:b4:d4:53:7c', '18:64:72:1d:84:21', '18:64:72:1d:90:43', '18:64:72:1d:90:42', '18:64:72:1d:90:41', 'f4:06:69:7d:3a:e3', 'e8:cc:18:de:56:0c', '18:64:72:1d:90:44', '18:64:72:23:02:e1', '18:64:72:23:02:e0', '18:64:72:23:02:e3', '18:64:72:23:02:e2', '18:64:72:23:02:e4', '18:64:72:1d:79:44', '18:64:72:36:16:04', '18:64:72:1d:83:03', '80:37:73:e9:d8:8b', '18:64:72:36:16:01', '18:64:72:36:16:00', '18:64:72:36:16:03', '18:64:72:36:16:02', '18:64:72:1d:79:42', '18:64:72:1d:79:43', '18:64:72:1d:79:41', 'ea:cc:18:de:56:0d', '18:64:72:1d:83:02', '18:64:72:1d:83:0

# Put the data into dataframe now
- Initialize everything with zero first. This will just create the structure of the dataframe for us. 
- We will programatically add Signal Strength to this DataFrame next. 
- Notice that there are 66 columns in total. 12 for distinct Cell Ids, 53 Distinct Wi-Fis and 1 label column in the end.

In [10]:
def createNewDataFrame(total_records, newColumns):
    newDF = pd.DataFrame(0, index = np.arange(total_records), columns=list(newColumns))  #using total_records and newColumns to make a new DF.
    return newDF

df = createNewDataFrame(total_records, newColumns)
df.head()                                                      

,8363,8362,7837,5273,7838,38173,7001,61148,8411,7002,...,18:64:72:1d:84:20,34:64:a9:87:dd:f4,18:64:72:1d:84:22,18:64:72:1d:84:23,18:64:72:1d:84:24,18:64:72:35:dc:c1,36:e2:fd:10:fd:fc,18:64:72:35:dc:c3,18:64:72:35:dc:c2,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df.head()

,8363,8362,7837,5273,7838,38173,7001,61148,8411,7002,...,18:64:72:1d:84:20,34:64:a9:87:dd:f4,18:64:72:1d:84:22,18:64:72:1d:84:23,18:64:72:1d:84:24,18:64:72:35:dc:c1,36:e2:fd:10:fd:fc,18:64:72:35:dc:c3,18:64:72:35:dc:c2,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Try writing to file directly

In [12]:
#Fill the dataframe with appropriate values and write it to the ouptput file
def fillDataIntoDataframe(df, outputFileNameWithPath, directoryPathOfInputFiles, dictDistinctCells, dictDistinctWifi):    
    fout = open(outputFileNameWithPath,"w")

    #Iterate over all the files in that directory
    index=-1
    for eachfile in os.listdir(directoryPathOfInputFiles):
        isNewFile = True
        local_count_cell=0; local_count_wifi=0
        labels = eachfile.split("_")[0]
        print labels                                       ##Can remove this print

        # Open each file in that directory and read it's line, one by one.
        fin = open(directoryPathOfInputFiles + eachfile,"r")
        next(fin)                                           #Skips the first line that is of no use for us.      
        for line in fin:
            splittedLine = re.split('\n', line)              #Splitted the line, use splttedLine[0] to access the line
            if splittedLine[0].find("EDGE Cell") != -1:
                flag = True
                arr = splittedLine[0].split(' ')
                if arr[3] == '65535':
                    continue
                else:
                    df[str(arr[3])][index] = int(arr[7])
                continue

            if splittedLine[0].find("WiFi:") != -1: #and flag == True:  #flag==true shows a switch  from Edge Cells to wifi
                arr = splittedLine[0].split(' ')
                df[arr[1]][index] = int(arr[-2])

            
            if splittedLine[0].find('_') != -1 and re.match(r'[0-9]+_[0-9]+_[0-9]+', splittedLine[0]): #This means a new block/record has begun
                if isNewFile == True:
                    isNewFile = False
                    index = index+1 #
                    next(fin); next(fin); next(fin)                         #Skip the next three lines to jump to the first EDGE CELL line
                else:                                                        #That means it is a switch from wifi to _ ie; we have entered a new record in the same file
                    df['label'][index] = labels                              #So we want to write the label of the previous index first
                    index = index +1
                    next(fin); next(fin); next(fin)
            #Inner for loop ends here.
        #This is also part of Outer for loop
        df['label'][index] = labels
        fin.close()
        #Outer for loop ends here.
    fout.close()
    return df
#The method definition ends here

wasFillingASuccess = False
outputFileNameWithPath = "C:/Users/Anshul/Desktop/Courses/Wireless/Final_Project/output1Directly.csv"
directoryPathOfInputFiles = "C:/Users/Anshul/Desktop/Courses/Wireless/Final_Project/Initial_Data/"
df = fillDataIntoDataframe(df, outputFileNameWithPath, directoryPathOfInputFiles, dictDistinctCells, dictDistinctWifi)
# print 'Was filling data and writing to output file successful? -->', wasFillingASuccess

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

C1044ANorth
C1044ASouth
C1044CDoor
C1044CWashroomB
C1044halldoor
C1044kitchenburner
C1044MainDoor


# Yes the Data Frame has been modified

In [13]:
df.head()

,8363,8362,7837,5273,7838,38173,7001,61148,8411,7002,...,18:64:72:1d:84:20,34:64:a9:87:dd:f4,18:64:72:1d:84:22,18:64:72:1d:84:23,18:64:72:1d:84:24,18:64:72:35:dc:c1,36:e2:fd:10:fd:fc,18:64:72:35:dc:c3,18:64:72:35:dc:c2,label
0,-91,0,0,0,-95,0,-97,-99,0,-95,...,-73,-90,-72,-72,-72,0,0,0,0,C1044ANorth
1,-91,0,0,0,-95,0,-99,-99,0,-95,...,-75,-90,-74,-74,-75,0,0,0,0,C1044ANorth
2,-91,0,0,0,-95,0,-99,-99,0,-95,...,-71,-90,-70,-70,-75,0,0,0,0,C1044ANorth
3,-91,0,0,0,-95,0,-99,-99,0,-95,...,-74,-91,-73,-72,-75,0,0,0,0,C1044ANorth
4,-91,0,0,0,-95,0,-99,-99,0,-95,...,-74,-89,-73,-73,-75,0,0,0,0,C1044ANorth


## Main Method

In [ ]:
import os
import re
import string
import pandas as pd
import numpy as np

if __name__ == "__main__":
    #Define the input file directory path and output file name with path.
    outputFileNameWithPath = "C:\Users\Anshul\Desktop\Courses\Wireless\Final_Project\output1Directly.csv"
    directoryPathOfInputFiles = "C:/Users/Anshul/Desktop/Courses/Wireless/Final_Project/Initial_Data/"
    dictDistinctCells = {}; dictDistinctWifi ={}
    
    #Get the max count of cells and wifi
    max_cell_count, max_wifi_count, total_records = getMaxCounts(directoryPathOfInputFiles)
    
    #Get the list of newColumns
    newColumns = makeColumnHeaders(dictDistinctCells, dictDistinctWifi)
    
    #Get back a new DataFrame of size total_records x length of newColumns. This will initially be filled with all zeros
    df = createNewDataFrame(total_records, newColumns)
    
    #Fill the dataframe with appropriate values and write it to the ouptput file
    wasFillingASuccess = False
    df = fillDataIntoDataframe(df, outputFileNameWithPath, directoryPathOfInputFiles, dictDistinctCells, dictDistinctWifi)


## Splitting into Training and testing

In [15]:
#1
import copy

dictDistinctLabels = {}; dictDistinctLabelsTrain80= {}; dictDistinctLabelsTest20= {}
totalColumns = len(df.columns)
lastColumn = df.ix[:,(totalColumns-1)].tolist()
for label in lastColumn:
    if label not in dictDistinctLabels:
        dictDistinctLabels[label] = 1
    else:
        dictDistinctLabels[label] = dictDistinctLabels[label] +1

# Print the dictDistinctLabels
print dictDistinctLabels

#Deep Copied to avoid any problems
dictDistinctLabelsDeepCopied = copy.deepcopy(dictDistinctLabels)

totalRowsInTrain80 = 0; totalRowsInTest20 = 0;
for key, value in dictDistinctLabelsDeepCopied.items():
    dictDistinctLabelsTrain80[key] = int(round(value * 0.8))
    totalRowsInTrain80 = totalRowsInTrain80 + int(round(value * 0.8))
    dictDistinctLabelsTest20[key] = int(round(value * 0.2))
    totalRowsInTest20 = totalRowsInTest20 + int(round(value * 0.2))
        

{'C1044kitchenburner': 613, 'C1044halldoor': 606, 'C1044ASouth': 614, 'C1044ANorth': 617, 'C1044MainDoor': 308, 'C1044CWashroomB': 310, 'C1044CDoor': 312}


## Made exact copies of DataFrame for TrainDf and TestDf
- Then we will delete 20% rows from TrainDf and 80% from TestDf

In [16]:
#2
TrainDF = df.copy(deep=True)
TestDF = df.copy(deep=True)

## Put 'delete' in the rows which you want to delete from TrainDF and TrainDF

In [22]:
#3
for i in range(len(lastColumn)):
    label = lastColumn[i]     
    if dictDistinctLabelsTrain80[label] > 0:
        TestDF['label'][i] = "delete"
        dictDistinctLabelsTrain80[label] = dictDistinctLabelsTrain80[label] -1
    elif dictDistinctLabelsTest20[label] > 0:
        TrainDF['label'][i] = "delete"
        dictDistinctLabelsTest20[label] = dictDistinctLabelsTest20[label] -1

TrainDF = TrainDF[TrainDF['label'] != 'delete']
TestDF = TestDF[TestDF['label'] != 'delete']
print dictDistinctLabelsTrain80

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


{'C1044kitchenburner': 0, 'C1044halldoor': 0, 'C1044ASouth': 0, 'C1044ANorth': 0, 'C1044MainDoor': 0, 'C1044CWashroomB': 0, 'C1044CDoor': 0}


## Yes, the Dataframes has been modified.
- Now the Training Dataframe contains 80% rows of the original dataframe by making sure that it has 80% data from all the records collected under each of the label.
- Similarly for TestDF. It contains only 20% of the original DataFrame now.

In [26]:
print 'Total Rows in Training DataFrame now is--> ', len(TrainDF)
print 'Total Rows in Testing DataFrame now is--> ', len(TestDF)
print 'Total Rows in original dataframe is--> ', len(df)
print 'Do TrainDF and TestDF sum up to the total rows in original DF?--> ', len(TrainDF) + len(TestDF) == len(df)

Total Rows in Training DataFrame now is-->  2704
Total Rows in Testing DataFrame now is-->  676
Total Rows in original dataframe is-->  3380
Do TrainDF and TestDF sum up to the total rows in original DF?-->  True


## Find the top few wifi contributor and add them sequentially to see if accuracy improves
- TO find the top contributors, vertically sumup the columns. The one with maximum absolute value is the one with maximum contribution.

In [27]:
#4
cellColumns = []
for i in range(len(dictDistinctCells)):
    cellColumns.append(i)

#5
import operator
listOfWifi=TrainDF.ix[:,cellColumns[-1]+1:len(TrainDF.columns)-1].columns.values.tolist()
lst=[]
tup=()
for wifi in listOfWifi:
    tup = (wifi, (abs(TrainDF[wifi].sum())))
    lst.append(tup)
lst.sort(key=operator.itemgetter(1))   ##Got a sorted list of wifi contributors
top3Wifi = []
top3Wifi.append(lst[-1][0]);   ## add the first one and keep adding one after the other to determine the accuracy of the model
top3Wifi.append(lst[-2][0]); 
top3Wifi.append(lst[-3][0]);
top3Wifi.append(lst[-4][0]);
top3Wifi.append(lst[-5][0]);
top3Wifi.append(lst[-6][0]);
top3Wifi.append(lst[-7][0]);
top3Wifi.append(lst[-8][0]);
top3Wifi.append(lst[-9][0]);
top3Wifi.append(lst[-10][0]);
# top3Wifi.append(lst[-11][0]);
# top3Wifi.append(lst[-12][0]);
# top3Wifi.append(lst[-13][0]);
# top3Wifi.append(lst[-14][0]);
# top3Wifi.append(lst[-15][0]);


#6
top3Index= TrainDF.ix[:,:].columns.values.tolist()
for wifi in top3Wifi:
    cellColumns.append(top3Index.index(wifi))
print cellColumns


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 59, 53, 50, 51, 24, 58, 21, 19, 22, 55]


# Train the model with Naive Bayes

In [28]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [30]:
featureMatrix = TrainDF.ix[:,cellColumns]   #Only Cellular
# featureMatrix = TrainDF.ix[:,0:len(TrainDF.columns)-1]   # all Cellular + all wifi


targetArray = TrainDF.ix[:, len(TrainDF.columns)-1].tolist()   #Only Cellular and at times with selected wifi
# targetArray = TrainDF.ix[:, len(TrainDF.columns)-1].tolist()    # all Cellular + all wifi


trainedModel = gnb.fit(featureMatrix, targetArray)
print 'Training Model Accuracy', trainedModel.score(featureMatrix, targetArray)*100

TestFeatureMatrix = TestDF.ix[:,cellColumns]    #Only Cellular
# TestFeatureMatrix = TestDF.ix[:,0:len(TrainDF.columns)-1]    # all Cellular + all wifi
y_pred = trainedModel.predict(TestFeatureMatrix)


mismatch = 0
TestTargetArray = TestDF.ix[:, len(TestDF.columns)-1].tolist()  #Only Cellular
# TestTargetArray = TestDF.ix[:, len(TestDF.columns)-1].tolist()   # all Cellular + all wifi
for i in range(len(TestTargetArray)):
    if y_pred[i] != TestTargetArray[i]:
        mismatch += 1
print 'Correctly Predicted/Testing Accuracy', str(100*(len(TestTargetArray)-mismatch)/(float)(len(TestTargetArray)))


Training Model Accuracy 88.7204142012
Correctly Predicted/Testing Accuracy 89.0532544379


# Conclusion -
- We got 53% accuracy after using only cellular
- After including 1 wi-Fi (thee most contributing one), it improved to 64%
- Kept adding Most contributing Wi-Fi and got some stability in accuracy after adding 6th Wifi from the list of most contributing wi-Fis. With 6 Wi-Fis, got the accuracy of 89.05%